In [1]:
# Xin quyen Google drive
from google.colab import drive
drive.mount('/content/ggdrive')

Mounted at /content/ggdrive


In [2]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.J5zsCys6Wp/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ioEowBjWiA/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.0c3EVfrYY6/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://archive.ubuntu.com/ubuntu 

In [3]:
import selenium

# print( selenium.__version__)


In [4]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

# url = "<url>"
# options = Options()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")

# options.headless = True
# from selenium.webdriver.chrome.service import Service
# browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Tạo đối tượng Options và thêm các tùy chọn cần thiết
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

# Tạo đối tượng driver
browser = webdriver.Chrome(options=options)

# Bạn có thể thực hiện các hoạt động với driver ở đây

# Đóng trình duyệt sau khi hoàn thành
# browser.quit()


In [5]:
import pandas as pd
from selenium.webdriver.common.by import By
from time import sleep

browser.get("https://finance.vietstock.vn/VNM/tai-chinh.htm?tab=BCTT")
sleep(5)

In [6]:
ChonKy_1= browser.find_element("xpath",'//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[1]/select[1]/option[8]')
ChonKy_1.click()

In [7]:
ChonNam_1= browser.find_element("xpath",'//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[1]/select[2]/option[1]')
ChonNam_1.click()


In [8]:
ChonKy_1= browser.find_element("xpath",'//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[1]/select[1]/option[6]')
ChonKy_1.click()

In [9]:
CTKD = browser.find_element("xpath",'//*[@id="tbl-data-BCTT-KQ"]')

table_CTKD = []

table_header_CTKD = CTKD.find_elements(By.XPATH,"thead/tr/th")

header_row_CTKD = []
for header in table_header_CTKD:
    header_row_CTKD.append(header.text)

table_data_CTKD = CTKD.find_elements(By.XPATH,"tbody/tr")
index_CTKD=[]
for row in table_data_CTKD:
    columns = row.find_elements(By.XPATH,"./td")
    table_row = []
    count=0
    for column in columns:
        if(count==0):
          index_CTKD.append(column.text)
        else:
          table_row.append(column.text)
        count+=1
    table_CTKD.append(table_row)

header_row_CTKD.pop(0)
df_CTKD = pd.DataFrame(table_CTKD, index = index_CTKD,columns =header_row_CTKD)
df_CTKD

,,,,2016,2017,2018,2019,2020,2021,2022
Giai đoạn,,,,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12
Hợp nhất,,,,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất
Kiểm toán,,,,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán
Công ty kiểm toán,,,,KPMG,KPMG,KPMG,KPMG,KPMG,KPMG,KPMG
Ý kiến kiểm toán,,,,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần
,,,,,,,,,,
Doanh thu thuần về bán hàng và cung cấp dịch vụ,,,,"46,794","51,041","52,562","56,318","59,636","60,919","59,956"
Giá vốn hàng bán,,,,"24,459","26,807","27,951","29,746","31,968","34,641","36,059"
Lợi nhuận gộp về bán hàng và cung cấp dịch vụ,,,,"22,336","24,234","24,611","26,572","27,669","26,278","23,897"
Doanh thu hoạt động tài chính,,,,723,816,760,807,"1,581","1,215","1,380"


In [10]:
CDKT = browser.find_element("xpath",'//*[@id="tbl-data-BCTT-CD"]')
# CDKT= browser.find_element("xpath",'///*[@id="tbl-data-BCTT-CD"]')

table_CDKT = []

table_header_CDKT = CDKT.find_elements(By.XPATH,"thead/tr/th")

header_row_CDKT = []
for header in table_header_CDKT:
    header_row_CDKT.append(header.text)

table_data_CDKT = CDKT.find_elements(By.XPATH,"tbody/tr")
index_CDKT=[]
for row in table_data_CDKT:
    columns = row.find_elements(By.XPATH,"./td")
    table_row = []
    count=0
    for column in columns:
        if(count==0):
          index_CDKT.append(column.text)
        else:
          table_row.append(column.text)
        count+=1
    table_CDKT.append(table_row)

header_row_CDKT.pop(0)
df_CDKT = pd.DataFrame(table_CDKT, index = index_CDKT,columns =header_row_CDKT)
df_CDKT

,,,,2016,2017,2018,2019,2020,2021,2022
Giai đoạn,,,,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12
Hợp nhất,,,,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất
Kiểm toán,,,,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán
Công ty kiểm toán,,,,KPMG,KPMG,KPMG,KPMG,KPMG,KPMG,KPMG
Ý kiến kiểm toán,,,,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần
,,,,,,,,,,
Tài sản ngắn hạn,,,,"18,674","20,307","20,560","24,722","29,666","36,110","31,560"
Tiền và các khoản tương đương tiền,,,,655,963,"1,523","2,665","2,111","2,349","2,300"
Các khoản đầu tư tài chính ngắn hạn,,,,"10,454","10,562","8,674","12,436","17,314","21,026","17,414"
Các khoản phải thu ngắn hạn,,,,"2,867","4,592","4,639","4,503","5,187","5,822","6,100"


In [11]:
CSTC = browser.find_element("xpath",'//*[@id="tbl-data-BCTT-CSTC"]')

table_CSTC = []

table_header_CSTC = CSTC.find_elements(By.XPATH,"thead/tr/th")

header_row_CSTC = []
for header in table_header_CSTC:
    header_row_CSTC.append(header.text)

table_data_CSTC = CSTC.find_elements(By.XPATH,"tbody/tr")
index_CSTC=[]
for row in table_data_CSTC:
    columns = row.find_elements(By.XPATH,"./td")
    table_row = []
    count=0
    for column in columns:
        if(count==0):
          index_CSTC.append(column.text)
        else:
          table_row.append(column.text)
        count+=1
    table_CSTC.append(table_row)

header_row_CSTC.pop(0)
df_CSTC = pd.DataFrame(table_CSTC, index = index_CSTC,columns =header_row_CSTC)
df_CSTC

,,,,2016,2017,2018,2019,2020,2021,2022
Giai đoạn,,,,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12,01/01-31/12
Hợp nhất,,,,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất,Hợp nhất
Kiểm toán,,,,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán,Kiểm toán
Công ty kiểm toán,,,,KPMG,KPMG,KPMG,KPMG,KPMG,KPMG,KPMG
Ý kiến kiểm toán,,,,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần,Chấp nhận toàn phần
,,,,,,,,,,
Thu nhập trên mỗi cổ phần của 4 quý gần nhất (EPS),,VNĐ,,"7,298.24","7,093.81","6,645.39","6,076.28","6,111.73","5,039.63","4,074.74"
Giá trị sổ sách của cổ phiếu (BVPS),,VNĐ,,"15,437.19","16,449.67","15,086.25","17,073.41","16,101.83","17,153.53","15,702.02"
Chỉ số giá thị trường trên thu nhập (P/E),,Lần,,17.21,29.41,18.06,19.17,17.8,17.14,18.68
Chỉ số giá thị trường trên giá trị sổ sách (P/B),,Lần,,8.14,12.68,7.95,6.82,6.76,5.04,4.85


In [12]:
df_CTKD.to_csv('CTKD_quy.csv',sep='\t',encoding='utf-16')
df_CDKT.to_csv('CDKT_quy.csv',sep='\t',encoding='utf-16')
df_CSTC.to_csv('CSTC_quy.csv',sep='\t',encoding='utf-16')

In [13]:
df_CTKD.to_csv('CTKD_nam.csv',sep='\t',encoding='utf-16')
df_CDKT.to_csv('CDKT_nam.csv',sep='\t',encoding='utf-16')
df_CSTC.to_csv('CSTC_nam.csv',sep='\t',encoding='utf-16')